In [8]:
import pandas as pd
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Parámetros de configuración
CSV_INPUT = "grecia_anotados/gr_combined.csv"         # Archivo CSV con columnas "id" y "mp_party"
JSON_OUTPUT = "tweets_griegos_anotados_finales.json"  # Archivo de salida en JSON
DELAY_BETWEEN_REQUESTS = 1         # Tiempo de retardo entre cada tweet
MAX_REPLIES =  50                  # Número máximo de respuestas a extraer por tweet (ajustable)

# Credenciales para login en Twitter
USERNAME = "marcosHern80439"
PASSWORD = "Servidor"

# 1. Configurar el WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless=new")       # Puedes quitarlo para debugear visualmente
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=chrome_options)


def login_twitter(driver, username, password):
    driver.get("https://twitter.com/i/flow/login")
    wait = WebDriverWait(driver, 20)
    # Espera y escribe el nombre de usuario (correo, teléfono o usuario)
    username_field = wait.until(
        EC.presence_of_element_located((By.XPATH, "//input[@name='text']"))
    )
    username_field.send_keys(username)
    username_field.send_keys(Keys.RETURN)
    time.sleep(3)  # Ajusta el tiempo si es necesario

    # En algunos casos aparece un botón "Siguiente"
    try:
        next_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and contains(., 'Siguiente')]"))
        )
        next_btn.click()
        time.sleep(3)
    except Exception as e:
        print("No se encontró el botón 'Siguiente':", e)

    # Espera el campo de contraseña y escribe la contraseña
    password_field = wait.until(
        EC.presence_of_element_located((By.XPATH, "//input[@name='password']"))
    )
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)
    time.sleep(5)  # Espera a que se complete el login


def extract_tweet_data(driver, tweet_id, max_attempts=3, max_replies=MAX_REPLIES):
    """
    Extrae el texto del tweet original, el usuario y hasta max_replies respuestas.
    Retorna un diccionario con 'tweet_text', 'user', y 'replies' (lista de diccionarios).
    """
    url = f"https://twitter.com/i/web/status/{tweet_id}"
    attempt = 0
    while attempt < max_attempts:
        try:
            driver.get(url)
            wait = WebDriverWait(driver, 20)
            # Extraer tweet original
            tweet_text_el = wait.until(
                EC.presence_of_element_located((By.XPATH, '//div[@data-testid="tweetText"]'))
            )
            tweet_text = tweet_text_el.text.strip()
            user_el = wait.until(
                EC.presence_of_element_located((By.XPATH, '//div[@data-testid="User-Name"]'))
            )
            user = user_el.text.strip()

            # Scroll para cargar respuestas
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            # Extraer respuestas
            replies = []
            # Cada tweet en la conversación es un <article data-testid="tweet">
            articles = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
            # Ignorar el primero (tweet original)
            for art in articles[1: max_replies+1]:
                try:
                    reply_text = art.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text.strip()
                    reply_user = art.find_element(By.XPATH, './/div[@data-testid="User-Name"]').text.strip()
                    replies.append({'user': reply_user, 'reply': reply_text})
                except Exception:
                    continue

            return {'tweet_text': tweet_text, 'user': user, 'replies': replies}

        except Exception as e:
            attempt += 1
            print(f"Intento {attempt} falló para tweet {tweet_id}. Error: {e}")
    # En caso de fallo total, retornar mensajes de error
    return {
        'tweet_text': f"Error al extraer tweet {tweet_id} tras {max_attempts} intentos.",
        'user': f"Error al extraer usuario de {tweet_id} tras {max_attempts} intentos.",
        'replies': []
    }

print("empezamos")
# 2. Iniciar sesión en Twitter
login_twitter(driver, USERNAME, PASSWORD)
print("Login completado, sesión activa.")

# 3. Cargar el CSV y validar columnas

df = pd.read_csv(CSV_INPUT)
if 'id' not in df.columns:
    raise Exception("El archivo CSV debe tener las columnas 'id' y 'mp_party'")
rows = df.to_dict('records')

# 4. Recorrer el CSV, extraer datos y crear la estructura JSON
results = []
for row in rows:
    tweet_id = row['id']
    full_text = row['full_text']
    print(f"Procesando tweet ID: {tweet_id}")
    data = extract_tweet_data(driver, tweet_id)
    entry = {
        'user': data['user'],
        'tweet': full_text,
        'replies': data['replies']
    }
    results.append(entry)
    print(f"Resultados parciales para {tweet_id}: {entry}")
    time.sleep(DELAY_BETWEEN_REQUESTS)

# Cerrar el driver

driver.quit()

# 5. Guardar la salida en un archivo JSON

with open(JSON_OUTPUT, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"Resultados guardados en {JSON_OUTPUT}")


empezamos
No se encontró el botón 'Siguiente': Message: 
Stacktrace:
#0 0x60a33ff55cea <unknown>
#1 0x60a33fa065f0 <unknown>
#2 0x60a33fa57a33 <unknown>
#3 0x60a33fa57c21 <unknown>
#4 0x60a33faa6274 <unknown>
#5 0x60a33fa7d68d <unknown>
#6 0x60a33faa3660 <unknown>
#7 0x60a33fa7d433 <unknown>
#8 0x60a33fa49ea3 <unknown>
#9 0x60a33fa4ab01 <unknown>
#10 0x60a33ff1ab3b <unknown>
#11 0x60a33ff1ea21 <unknown>
#12 0x60a33ff01c32 <unknown>
#13 0x60a33ff1f594 <unknown>
#14 0x60a33fee5eef <unknown>
#15 0x60a33ff43d98 <unknown>
#16 0x60a33ff43f76 <unknown>
#17 0x60a33ff54b36 <unknown>
#18 0x70f35a094ac3 <unknown>

Login completado, sesión activa.
Procesando tweet ID: 1349005840745984007
Resultados parciales para 1349005840745984007: {'user': 'Giannis Plakiotakis\n@G_Plakiotakis', 'tweet': 'Την Τετάρτη το πρωί, στις 9:15 στον ΣΚΑΪ και την εκπομπή "Σήμερα" #Πλακιωτάκης #ΣΚΑΪ https://t.co/PynuyD0upJ', 'replies': []}
Procesando tweet ID: 1352200868943245313
Resultados parciales para 13522008689432453